# Fuzzy Tsukamoto dan Mamdani

## Import Library yang dibutuhkan

In [1]:
import pandas as pd
import json
from collections import defaultdict
from scipy.integrate import quad

## Penentuan himpunan fuzzy

In [2]:
#nama file data latih ditulis disini
df = pd.read_csv('DataProduksiBenih_Penghujan_New.csv')

# print(df.loc[0]) untuk cari data csv nya by row

data = {
    "luas_lahan": {
        "banyak": max(df.luas_lahan),
        "sedikit": min(df.luas_lahan),
    },
    "benih_sumber": {
        "banyak": max(df.benih_sumber),
        "sedikit": min(df.benih_sumber),
    },
    "jumlah_pupuk": {
        "banyak": max(df.jumlah_pupuk),
        "sedikit": min(df.jumlah_pupuk),
    },
    "hasil_produksi": {
        "banyak": max(df.hasil_produksi),
        "sedikit": min(df.hasil_produksi)
    }
}

test = pd.DataFrame(data)

print("Data produksi benih \n")
print(test)


Data produksi benih 

         luas_lahan  benih_sumber  jumlah_pupuk  hasil_produksi
banyak            8           320          4800           30000
sedikit           5           200          3000           17500


## Data uji yang digunakan

In [3]:
kolom_uji = ['luas_lahan', 'benih_sumber', 'jumlah_pupuk']

#nama file data uji ditulis disini
file = "DataUji.csv"

data_uji = pd.read_csv(file, skipinitialspace=True, usecols=kolom_uji)
data_uji.index += 1 

kolom_uji = ['hasil_produksi']
hasil_produksi = pd.read_csv(file, skipinitialspace=True, usecols=kolom_uji)
hasil_produksi.index += 1 

print("Data uji pada sistem\n")
print(data_uji)

Data uji pada sistem

   luas_lahan  benih_sumber  jumlah_pupuk
1           7           280          4200
2           7           280          4200


## Fuzzifikasi derajat keanggotaan 

In [4]:
def banyak(b,a,x):
    if(x<=a):
        nilai = 0
    elif(x>a and x<b):
        nilai = (x-a)/(b-a)
    elif(x>=b):
        nilai = 1

    return nilai

In [5]:
def sedikit(b,a,x):
    if(x<=a):
        nilai = 1
    elif(x>a and x<b):
        nilai = (b-x)/(b-a)
    elif(x>=b):
        nilai = 0

    return nilai

In [6]:
fuzz_luas_lahan = defaultdict(dict)
fuzz_benih_sumber = defaultdict(dict)
fuzz_jumlah_pupuk = defaultdict(dict)

for x in range(len(data_uji)):
    x += 1
    #Penyimpanan Data, gunakan var ini untuk fuzzifikasi
    fuzz_luas_lahan[x]["banyak"] = banyak(data["luas_lahan"]["banyak"],data["luas_lahan"]["sedikit"],data_uji.loc[x].luas_lahan)
    fuzz_luas_lahan[x]["sedikit"] = sedikit(data["luas_lahan"]["banyak"],data["luas_lahan"]["sedikit"],data_uji.loc[x].luas_lahan)
    
    fuzz_benih_sumber[x]["banyak"] = banyak(data["benih_sumber"]["banyak"],data["benih_sumber"]["sedikit"],data_uji.loc[x].benih_sumber)
    fuzz_benih_sumber[x]["sedikit"] = sedikit(data["benih_sumber"]["banyak"],data["benih_sumber"]["sedikit"],data_uji.loc[x].benih_sumber)
    
    fuzz_jumlah_pupuk[x]["banyak"] = banyak(data["jumlah_pupuk"]["banyak"],data["jumlah_pupuk"]["sedikit"],data_uji.loc[x].jumlah_pupuk)
    fuzz_jumlah_pupuk[x]["sedikit"] = sedikit(data["jumlah_pupuk"]["banyak"],data["jumlah_pupuk"]["sedikit"],data_uji.loc[x].jumlah_pupuk)
    
    fuzz_raw = {
        "banyak": {
            "luas_lahan": fuzz_luas_lahan[x]["banyak"],
            "benih_sumber": fuzz_benih_sumber[x]["banyak"],
            "jumlah_pupuk": fuzz_jumlah_pupuk[x]["banyak"]
        },
        "sedikit": {
            "luas_lahan": fuzz_luas_lahan[x]["sedikit"],
            "benih_sumber": fuzz_benih_sumber[x]["sedikit"],
            "jumlah_pupuk": fuzz_jumlah_pupuk[x]["sedikit"]
        }
    }
    print("Hasil fuzzifikasi derajat keanggotaan data uji ke",x,"\n")
    print(pd.DataFrame(fuzz_raw))
    print("\n")


Hasil fuzzifikasi derajat keanggotaan data uji ke 1 

                banyak   sedikit
luas_lahan    0.666667  0.333333
benih_sumber  0.666667  0.333333
jumlah_pupuk  0.666667  0.333333


Hasil fuzzifikasi derajat keanggotaan data uji ke 2 

                banyak   sedikit
luas_lahan    0.666667  0.333333
benih_sumber  0.666667  0.333333
jumlah_pupuk  0.666667  0.333333




## Fungsi Implikasi Inferensi Aturan

#### [R1] IF luas lahan sedikit AND jumlah sumber benih sedikit AND jumlah pupuk sedikit THEN hasil produksi benih sedikit
#### [R2] IF luas lahan sedikit AND jumlah sumber benih banyak AND jumlah pupuk banyak THEN hasil produksi benih sedikit
#### [R3] IF luas lahan sedikit AND jumlah sumber benih banyak AND jumlah pupuk sedikit THEN hasil produksi benih sedikit
#### [R4] IF luas lahan banyak AND jumlah sumber benih sedikit AND jumlah pupuk banyak THEN hasil produksi benih banyak
#### [R5] IF luas lahan banyak AND jumlah sumber benih sedikit AND jumlah pupuk sedikit THEN hasil produksi benih banyak
#### [R6] IF luas lahan banyak AND jumlah sumber benih banyak AND jumlah pupuk banyak THEN hasil produksi benih banyak

In [7]:
#panjang aturan berdasarkan isi list
# contoh -> [rule1, rule2, ... , rule6]

aturan_raw = {
    "luas_lahan":["sedikit","sedikit","sedikit","banyak","banyak","banyak"],
    "benih_sumber": ["sedikit","banyak","banyak","sedikit","sedikit","banyak"],
    "jumlah_pupuk": ["sedikit","banyak","sedikit","banyak","sedikit","banyak"],
    "hasil_produksi": ["sedikit","sedikit","sedikit","banyak","banyak","banyak"]
}

aturan = pd.DataFrame(aturan_raw)
aturan.index += 1 

print("Tabel kebenaran berdasarkan aturan \n")
print(aturan)

Tabel kebenaran berdasarkan aturan 

  luas_lahan benih_sumber jumlah_pupuk hasil_produksi
1    sedikit      sedikit      sedikit        sedikit
2    sedikit       banyak       banyak        sedikit
3    sedikit       banyak      sedikit        sedikit
4     banyak      sedikit       banyak         banyak
5     banyak      sedikit      sedikit         banyak
6     banyak       banyak       banyak         banyak


In [8]:
def agregasi_naik(b,a,alfa):
    nilai = alfa*(b-a) + a
    return nilai

In [9]:
def agregasi_turun(b,a,alfa):
    nilai = b - (alfa*(b-a))
    return nilai

## Menentukan nilai alpha dan z

In [10]:
alpha_sedikit = defaultdict(list)
alpha_banyak = defaultdict(list)

alpha = defaultdict(list)
z = defaultdict(list)

for i in range(len(data_uji)):
    i += 1
    for j in range(len(aturan)):
        j += 1
        #print(i,j)

        kondisi1 = aturan.loc[j].luas_lahan
        kondisi2 = aturan.loc[j].benih_sumber
        kondisi3 = aturan.loc[j].jumlah_pupuk
        kondisi4 = aturan.loc[j].hasil_produksi
        
        alpha_temp = min(fuzz_luas_lahan[i][kondisi1], fuzz_benih_sumber[i][kondisi2], fuzz_jumlah_pupuk[i][kondisi3])
        alpha[i].append(alpha_temp)
        
        if(kondisi4 == "banyak"):
            z_temp = agregasi_naik(data["hasil_produksi"]["banyak"], data["hasil_produksi"]["sedikit"], alpha_temp)
            alpha_banyak[i].append(alpha_temp)
        elif(kondisi4 == "sedikit"):
            z_temp = agregasi_turun(data["hasil_produksi"]["banyak"], data["hasil_produksi"]["sedikit"], alpha_temp)
            alpha_sedikit[i].append(alpha_temp)
            
        z[i].append(z_temp)
        #print(alpha_temp, z_temp)
    #print("\n")

In [11]:
for x in range(len(data_uji)):
    x += 1
    data_alpha_z = {
        "nilai_alpha": alpha[x],
        "nilai_z": z[x]
    }
    draw_alpha_z = pd.DataFrame(data_alpha_z)
    draw_alpha_z.index += 1
    print("Tabel nilai alpha dan Z pada data uji ke -",x)
    print(draw_alpha_z, "\n")

Tabel nilai alpha dan Z pada data uji ke - 1
   nilai_alpha       nilai_z
1     0.333333  25833.333333
2     0.333333  25833.333333
3     0.333333  25833.333333
4     0.333333  21666.666667
5     0.333333  21666.666667
6     0.666667  25833.333333 

Tabel nilai alpha dan Z pada data uji ke - 2
   nilai_alpha       nilai_z
1     0.333333  25833.333333
2     0.333333  25833.333333
3     0.333333  25833.333333
4     0.333333  21666.666667
5     0.333333  21666.666667
6     0.666667  25833.333333 



## Komposisi aturan untuk perhitungan mamdani

In [12]:
def komposisiAturan(x, y):
    komposisi={
        "a1": agregasi_naik(data["hasil_produksi"]["banyak"], data["hasil_produksi"]["sedikit"], x),
        "a2": agregasi_naik(data["hasil_produksi"]["banyak"], data["hasil_produksi"]["sedikit"], y)
    }
    return komposisi

In [13]:
a_min = []
a_max = []
komposisi_a1 = []
komposisi_a2 = []

#komposisi = komposisiAturan(a_min, a_max)

for x in range(len(data_uji)):
    x += 1
    
    a_min.append(max(alpha_sedikit[x]))
    a_max.append(max(alpha_banyak[x]))
    
    komposisi_temp = komposisiAturan(a_min[x-1], a_max[x-1])
    
    komposisi_a1.append(komposisi_temp["a1"])
    komposisi_a2.append(komposisi_temp["a2"])

komposisi_raw = {
    "a_min": a_min,
    "a_max": a_max,
    "a1": komposisi_a1,
    "a2": komposisi_a2
}

draw_komposisi = pd.DataFrame(komposisi_raw)
draw_komposisi.index += 1

print(draw_komposisi)

      a_min     a_max            a1            a2
1  0.333333  0.666667  21666.666667  25833.333333
2  0.333333  0.666667  21666.666667  25833.333333


## Defuzzifikasi Tsukamoto

In [14]:
defuzz_tsukamoto = []

for i in range(len(data_uji)):
    i += 1
    defuzz_sum = 0
    
    for j in range(len(alpha[1])):
        defuzz_sum += alpha[i][j]*z[i][j]
    
    result_tsukamoto = int(defuzz_sum/sum(alpha[i]))
    defuzz_tsukamoto.append(result_tsukamoto)

## Defuzzifikasi Mamdani

### Mencari Nilai Momen

In [15]:
def nilaiMomen(m_a_min, m_a_max, m_a1, m_a2):
    momen = []
    if(m_a1 < m_a2):
        #Momen 1, f1 persamaan integral, a1 merupakan variable
        f1 = lambda z1,a1 : a1*z1
        a1 = m_a_min
        m1 = quad(f1, 0, m_a1, args=(a1))
        momen.append(m1[0])

        #Momen 2, f2 persamaan integral, a2 dan b2 merupakan variable
        f2 = lambda z2,a2,b2 : (z2-a2)/b2*z2
        a2 = data["hasil_produksi"]["sedikit"]
        b2 = data["hasil_produksi"]["banyak"]-data["hasil_produksi"]["sedikit"]
        m2 = quad(f2, m_a1, m_a2, args=(a2,b2))
        momen.append(m2[0])

        #Momen 3, f3 persamaan integral, a3 merupakan variable
        f3 = lambda z3,a3 : a3*z3
        a3 = m_a_max
        m3 = quad(f3, m_a2, data["hasil_produksi"]["banyak"], args=(a3))
        momen.append(m3[0])
    else:
        #Momen 1, f1 persamaan integral, a1 merupakan variable
        f1 = lambda z1,a1 : a1*z1
        a1 = m_a_min
        m1 = quad(f1, 0, m_a2, args=(a1))
        momen.append(m1[0])

        #Momen 2, f2 persamaan integral, a2 dan b2 merupakan variable
        f2 = lambda z2,a2,b2 : (z2-a2)/b2*z2
        a2 = data["hasil_produksi"]["sedikit"]
        b2 = data["hasil_produksi"]["banyak"]-data["hasil_produksi"]["sedikit"]
        m2 = quad(f2, m_a2, m_a1, args=(a2,b2))
        momen.append(m2[0])

        #Momen 3, f3 persamaan integral, a3 merupakan variable
        f3 = lambda z3,a3 : a3*z3
        a3 = m_a_max
        m3 = quad(f3, m_a1, data["hasil_produksi"]["banyak"], args=(a3))
        momen.append(m3[0])
    return momen

### Mencari Nilai Luas

In [16]:
def nilaiLuas(l_a_min,l_a_max,l_a1,l_a2):
    luas = []
    if(l_a1 < l_a2):
        luas.append(l_a1*l_a_min)
        luas.append((l_a_min+l_a_max)*((l_a2-l_a1)/2))
        luas.append((data["hasil_produksi"]["banyak"]-l_a2)*l_a_max)
    else:
        luas.append(l_a2*l_a_min)
        luas.append((l_a_min+l_a_max)*((l_a1-l_a2)/2))
        luas.append((data["hasil_produksi"]["banyak"]-l_a1)*l_a_max)
    return luas

In [17]:
#print(a_max)
defuzz_mamdani = []
for x in range(len(data_uji)):
    test1 = a_min[x]
    test2 = a_max[x]
    
    momen = nilaiMomen(a_min[x], a_max[x], komposisi_a1[x], komposisi_a2[x])
    luas = nilaiLuas(a_min[x], a_max[x], komposisi_a1[x], komposisi_a2[x])
    #print(momen,luas)
    #print(luas)
    result_mamdani = int(sum(momen)/sum(luas))
    defuzz_mamdani.append(result_mamdani)
    #print(result_mamdani)
    #print("\n")

## Hasil Prediksi

In [18]:
data_prediksi = {
    "luas_lahan": data_uji.luas_lahan,
    "benih_sumber": data_uji.benih_sumber,
    "jumlah_pupuk": data_uji.jumlah_pupuk,
    "jumlah_produksi": hasil_produksi.hasil_produksi,
    "prediksi_tsukamoto": defuzz_tsukamoto,
    "prediksi_mamdani": defuzz_mamdani
}

#print(data_prediksi)
hasil_prediksi = pd.DataFrame(data_prediksi)
print(hasil_prediksi)

   luas_lahan  benih_sumber  jumlah_pupuk  jumlah_produksi  \
1           7           280          4200            25585   
2           7           280          4200            25660   

   prediksi_tsukamoto  prediksi_mamdani  
1               24642             17027  
2               24642             17027  


## Perhitungan MAPE (Mean Absolute Percentage Error)

In [19]:
#print(hasil_produksi)
mape_tsukamoto = 0
mape_mamdani = 0
temp_tsukamoto = 0
temp_mamdani = 0

i = 0

for x in hasil_produksi.hasil_produksi:
    temp_tsukamoto += (x - defuzz_tsukamoto[i])/x
    temp_mamdani += (x - defuzz_mamdani[i])/x
    i += 1

mape_tsukamoto = abs(temp_tsukamoto)*100/len(hasil_produksi)
print("Hasil Perhitungan MAPE Tsukamoto :",round(mape_tsukamoto,2),"%")

mape_mamdani = abs(temp_mamdani)*100/len(hasil_produksi)
print("Hasil Perhitungan MAPE Mamdani :",round(mape_mamdani,2),"%")

Hasil Perhitungan MAPE Tsukamoto : 3.83 %
Hasil Perhitungan MAPE Mamdani : 33.55 %
